In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
set.seed(88888888) # maximum luck

library(DESeq2)
library(plotly)
library(ggplot2)
library(viridis)
library(magrittr)
library(pheatmap)
library(DescTools)
library(pdfCluster)
library(RColorBrewer)
library(SummarizedExperiment)
library(caret)
library(class)
library(htmlwidgets)

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Attaching package: ‘S4Vectors’

The following object is masked from ‘package:utils’:

    findMatches

The following objects are masked from ‘package:base’:

    expand.grid, I, unname

Loading required package: IRanges
Loading required package: GenomicRanges
Loading required package: GenomeInfoDb
Loading require

In [3]:
%%R
OUT_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/output/"
N_FOLDS <- 10

In [4]:
%%R
vst.counts <- readRDS(paste(OUT_DIR, "vst_counts030_time_cross_50.Rds", sep = ""))
print(vst.counts)

class: DESeqTransform 
dim: 9297 68 
metadata(1): version
assays(1): ''
rownames(9297): ENSMUSG00000000001 ENSMUSG00000000028 ...
  ENSMUSG00000118462 ENSMUSG00000118541
rowData names(4): baseMean baseVar allZero dispFit
colnames(68): SRR7817650 SRR7817612 ... SRR7817658 SRR7817667
colData names(3): Sample Dose sizeFactor


In [5]:
%%R
sample <- colData(vst.counts)
# print(sample)

In [6]:
%%R
vst.count.mtx <-
   vst.counts %>% SummarizedExperiment::assay() %>% as.data.frame()
print(length(vst.count.mtx))
 head(vst.count.mtx)

[1] 68
                   SRR7817650 SRR7817612 SRR3593524 SRR7817684 SRR7817697
ENSMUSG00000000001  15.372724  15.489787  14.859455  15.613249  15.278217
ENSMUSG00000000028   7.061426   6.441777   6.706037   6.708989   6.744838
ENSMUSG00000000049  20.478603  20.160850  20.540259  20.467407  20.266653
ENSMUSG00000000058   7.314136   7.794396   7.694075   7.461696   7.609270
ENSMUSG00000000085  10.524518   9.901466  10.184849   9.384417  10.169379
ENSMUSG00000000088  16.429167  16.780332  15.763619  16.599944  16.634191
                   SRR3593525 SRR7817672 SRR7817664 SRR4317610 SRR7817637
ENSMUSG00000000001  14.884786  15.255230  15.323637  15.143300  15.564530
ENSMUSG00000000028   6.621081   6.960122   6.520671   7.070111   6.613111
ENSMUSG00000000049  20.571958  20.057895  20.341317  20.596203  20.495006
ENSMUSG00000000058   7.805331   7.826987   7.822231   9.123771   8.373000
ENSMUSG00000000085  10.142078   9.989063   9.529526   9.912657  10.777504
ENSMUSG00000000088  15.807618  

In [8]:
%%R
tcdd_dose_detail.vec <-
   readRDS(paste(OUT_DIR, "tcdd_dose_detail_vec030_time_cross_50.Rds", sep = ""))
print(tcdd_dose_detail.vec)

 [1]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
[26]  0  0  0  0  0  0  0  0  0 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
[51] 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30


In [9]:
%%R
rxn2ensembls.nls <-
   readRDS(paste(OUT_DIR, "rxn2ensembls_nls030_time_cross_50.Rds", sep = ""))
print(length(rxn2ensembls.nls))
print(rxn2ensembls.nls)

[1] 9825
$`R-MMU-170666`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSMUSG00000057614"

$`R-MMU-170671`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSMUSG00000057614"

$`R-MMU-170674`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSM

 [46] "ENSMUSG00000028777" "ENSMUSG00000028950" "ENSMUSG00000028971"
 [49] "ENSMUSG00000029064" "ENSMUSG00000029072" "ENSMUSG00000029236"
 [52] "ENSMUSG00000029371" "ENSMUSG00000029372" "ENSMUSG00000029373"
 [55] "ENSMUSG00000029379" "ENSMUSG00000029380" "ENSMUSG00000029417"
 [58] "ENSMUSG00000029530" "ENSMUSG00000029663" "ENSMUSG00000029713"
 [61] "ENSMUSG00000029819" "ENSMUSG00000031778" "ENSMUSG00000031861"
 [64] "ENSMUSG00000031980" "ENSMUSG00000032192" "ENSMUSG00000032562"
 [67] "ENSMUSG00000032766" "ENSMUSG00000033717" "ENSMUSG00000034117"
 [70] "ENSMUSG00000034855" "ENSMUSG00000035042" "ENSMUSG00000035383"
 [73] "ENSMUSG00000035431" "ENSMUSG00000035448" "ENSMUSG00000036353"
 [76] "ENSMUSG00000036362" "ENSMUSG00000036381" "ENSMUSG00000036402"
 [79] "ENSMUSG00000036437" "ENSMUSG00000036832" "ENSMUSG00000037010"
 [82] "ENSMUSG00000037014" "ENSMUSG00000037140" "ENSMUSG00000037346"
 [85] "ENSMUSG00000037944" "ENSMUSG00000038607" "ENSMUSG00000038668"
 [88] "ENSMUSG00000038811" "ENSMUS

 [4] "ENSMUSG00000019775" "ENSMUSG00000020599" "ENSMUSG00000021219"
 [7] "ENSMUSG00000024186" "ENSMUSG00000026357" "ENSMUSG00000026358"
[10] "ENSMUSG00000026475" "ENSMUSG00000026527" "ENSMUSG00000026678"
[13] "ENSMUSG00000028777" "ENSMUSG00000029101" "ENSMUSG00000032562"
[16] "ENSMUSG00000038530" "ENSMUSG00000042671" "ENSMUSG00000051079"
[19] "ENSMUSG00000052087" "ENSMUSG00000057614" "ENSMUSG00000059810"

$`R-MMU-8982019`
 [1] "ENSMUSG00000000001" "ENSMUSG00000002458" "ENSMUSG00000002459"
 [4] "ENSMUSG00000019775" "ENSMUSG00000020599" "ENSMUSG00000021219"
 [7] "ENSMUSG00000024186" "ENSMUSG00000026357" "ENSMUSG00000026358"
[10] "ENSMUSG00000026475" "ENSMUSG00000026527" "ENSMUSG00000026678"
[13] "ENSMUSG00000028777" "ENSMUSG00000029101" "ENSMUSG00000032562"
[16] "ENSMUSG00000038530" "ENSMUSG00000042671" "ENSMUSG00000051079"
[19] "ENSMUSG00000052087" "ENSMUSG00000057614" "ENSMUSG00000059810"

$`R-MMU-8982020`
 [1] "ENSMUSG00000000001" "ENSMUSG00000002458" "ENSMUSG00000002459"
 [4] "ENSMUS

 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000024325" "ENSMUSG00000025577" "ENSMUSG00000025578"
 [7] "ENSMUSG00000026021" "ENSMUSG00000026484" "ENSMUSG00000026739"
[10] "ENSMUSG00000028796" "ENSMUSG00000037652" "ENSMUSG00000039989"
[13] "ENSMUSG00000040669"

$`R-MMU-4570463`
 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000020265" "ENSMUSG00000020738" "ENSMUSG00000024325"
 [7] "ENSMUSG00000025577" "ENSMUSG00000025578" "ENSMUSG00000026484"
[10] "ENSMUSG00000026739" "ENSMUSG00000028796" "ENSMUSG00000031347"
[13] "ENSMUSG00000037652" "ENSMUSG00000039989" "ENSMUSG00000040669"

$`R-MMU-4570499`
 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000020738" "ENSMUSG00000021546" "ENSMUSG00000024325"
 [7] "ENSMUSG00000025577" "ENSMUSG00000025578" "ENSMUSG00000026484"
[10] "ENSMUSG00000026739" "ENSMUSG00000028796" "ENSMUSG00000037652"
[13] "ENSMUSG00000039989" "ENSMUSG00000040669"

$`R-MM

 [1] "ENSMUSG00000000126" "ENSMUSG00000009900" "ENSMUSG00000010797"
 [4] "ENSMUSG00000012282" "ENSMUSG00000015957" "ENSMUSG00000018486"
 [7] "ENSMUSG00000021994" "ENSMUSG00000022382" "ENSMUSG00000022996"
[10] "ENSMUSG00000022997" "ENSMUSG00000026167" "ENSMUSG00000027840"
[13] "ENSMUSG00000028173" "ENSMUSG00000029671" "ENSMUSG00000030093"
[16] "ENSMUSG00000030170" "ENSMUSG00000033227" "ENSMUSG00000036856"
[19] "ENSMUSG00000036961"

$`R-MMU-3781943`
 [1] "ENSMUSG00000000126" "ENSMUSG00000009900" "ENSMUSG00000010797"
 [4] "ENSMUSG00000012282" "ENSMUSG00000015957" "ENSMUSG00000018486"
 [7] "ENSMUSG00000021994" "ENSMUSG00000022382" "ENSMUSG00000022996"
[10] "ENSMUSG00000022997" "ENSMUSG00000026167" "ENSMUSG00000027840"
[13] "ENSMUSG00000028173" "ENSMUSG00000029671" "ENSMUSG00000030093"
[16] "ENSMUSG00000030170" "ENSMUSG00000033227" "ENSMUSG00000036856"
[19] "ENSMUSG00000036961"

$`R-MMU-205328`
[1] "ENSMUSG00000000127" "ENSMUSG00000005672" "ENSMUSG00000019966"
[4] "ENSMUSG00000032113" "ENSM

...

[1] "ENSMUSG00000024387" "ENSMUSG00000030688" "ENSMUSG00000046707"
[4] "ENSMUSG00000074698"

$`R-MMU-8948039`
[1] "ENSMUSG00000024387" "ENSMUSG00000025040" "ENSMUSG00000046707"
[4] "ENSMUSG00000074698"

$`R-MMU-5246478`
[1] "ENSMUSG00000024391"

$`R-MMU-3371353`
[1] "ENSMUSG00000024401" "ENSMUSG00000030341" "ENSMUSG00000037316"

$`R-MMU-3371385`
[1] "ENSMUSG00000024401" "ENSMUSG00000052593"

$`R-MMU-5626953`
[1] "ENSMUSG00000024401" "ENSMUSG00000030341" "ENSMUSG00000040687"

$`R-MMU-5626988`
[1] "ENSMUSG00000024401" "ENSMUSG00000028245" "ENSMUSG00000030341"

$`R-MMU-5668404`
[1] "ENSMUSG00000024401" "ENSMUSG00000028599"

$`R-MMU-5668467`
[1] "ENSMUSG00000024401" "ENSMUSG00000028599"

$`R-MMU-83660`
[1] "ENSMUSG00000024401" "ENSMUSG00000030341" "ENSMUSG00000037316"

$`R-MMU-8863101`
[1] "ENSMUSG00000024401" "ENSMUSG00000027366" "ENSMUSG00000035206"

$`R-MMU-9714924`
[1] "ENSMUSG00000024401"

$`R-MMU-9714959`
[1] "ENSMUSG00000024401"

$`R-MMU-5669097`
[1] "ENSMUSG00000024402" "ENSMUSG000

[10] "ENSMUSG00000049010" "ENSMUSG00000049098" "ENSMUSG00000049362"
[13] "ENSMUSG00000049648" "ENSMUSG00000050028" "ENSMUSG00000051509"
[16] "ENSMUSG00000055088" "ENSMUSG00000055610" "ENSMUSG00000056822"
[19] "ENSMUSG00000057179" "ENSMUSG00000059043" "ENSMUSG00000059279"
[22] "ENSMUSG00000059473" "ENSMUSG00000059610" "ENSMUSG00000059623"
[25] "ENSMUSG00000060254" "ENSMUSG00000061210" "ENSMUSG00000061614"
[28] "ENSMUSG00000062128" "ENSMUSG00000063549" "ENSMUSG00000063881"
[31] "ENSMUSG00000064121" "ENSMUSG00000064252" "ENSMUSG00000066240"
[34] "ENSMUSG00000068806" "ENSMUSG00000070423" "ENSMUSG00000071510"
[37] "ENSMUSG00000072707" "ENSMUSG00000073998" "ENSMUSG00000075196"
[40] "ENSMUSG00000075202" "ENSMUSG00000093839" "ENSMUSG00000093884"
[43] "ENSMUSG00000094140" "ENSMUSG00000094891" "ENSMUSG00000096169"
[46] "ENSMUSG00000108891" "ENSMUSG00000108908"

$`R-MMU-1855154`
[1] "ENSMUSG00000024525" "ENSMUSG00000027531"

$`R-MMU-1855210`
[1] "ENSMUSG00000024525" "ENSMUSG00000027531"

$`R-MMU-

[1] "ENSMUSG00000024789" "ENSMUSG00000032175" "ENSMUSG00000034394"
[4] "ENSMUSG00000054263"

$`R-MMU-6784189`
[1] "ENSMUSG00000024789" "ENSMUSG00000032175" "ENSMUSG00000054263"

$`R-MMU-6785898`
[1] "ENSMUSG00000024789" "ENSMUSG00000030748"

$`R-MMU-8950723`
[1] "ENSMUSG00000024789" "ENSMUSG00000049093"

$`R-MMU-982775`
[1] "ENSMUSG00000024789" "ENSMUSG00000055737"

$`R-MMU-982792`
[1] "ENSMUSG00000024789" "ENSMUSG00000055737"

$`R-NUL-1067645`
[1] "ENSMUSG00000024789" "ENSMUSG00000032175"

$`R-NUL-879922`
[1] "ENSMUSG00000024789"

$`R-NUL-8950050`
[1] "ENSMUSG00000024789"

$`R-MMU-5693012`
[1] "ENSMUSG00000024793" "ENSMUSG00000050395"

$`R-MMU-390931`
[1] "ENSMUSG00000024798" "ENSMUSG00000026322" "ENSMUSG00000028747"

$`R-MMU-194698`
[1] "ENSMUSG00000024799"

$`R-MMU-448629`
[1] "ENSMUSG00000024810" "ENSMUSG00000026069"

$`R-MMU-2064406`
[1] "ENSMUSG00000024812" "ENSMUSG00000053110"

$`R-MMU-2064418`
[1] "ENSMUSG00000024812" "ENSMUSG00000027803"

$`R-MMU-2064421`
[1] "ENSMUSG000000248

[1] "ENSMUSG00000025094" "ENSMUSG00000036330"

$`R-MMU-264997`
[1] "ENSMUSG00000025130" "ENSMUSG00000035804"

$`R-MMU-9737780`
[1] "ENSMUSG00000025130" "ENSMUSG00000028158"

$`R-MMU-9817575`
[1] "ENSMUSG00000025130" "ENSMUSG00000057069"

$`R-MMU-9014293`
[1] "ENSMUSG00000025132" "ENSMUSG00000033220"

$`R-MMU-9014456`
[1] "ENSMUSG00000025132" "ENSMUSG00000030220" "ENSMUSG00000073433"
[4] "ENSMUSG00000073982"

$`R-MMU-9018754`
[1] "ENSMUSG00000025132" "ENSMUSG00000029204" "ENSMUSG00000030220"
[4] "ENSMUSG00000073433"

$`R-MMU-1483190`
[1] "ENSMUSG00000025137"

$`R-MMU-70664`
[1] "ENSMUSG00000025140"

$`R-MMU-418871`
[1] "ENSMUSG00000025151" "ENSMUSG00000025876"

$`R-MMU-9769235`
[1] "ENSMUSG00000025151" "ENSMUSG00000045903"

$`R-MMU-163733`
[1] "ENSMUSG00000025153"

$`R-MMU-163756`
[1] "ENSMUSG00000025153"

$`R-MMU-199202`
[1] "ENSMUSG00000025153" "ENSMUSG00000025894"

$`R-MMU-75872`
[1] "ENSMUSG00000025153"

$`R-MMU-5676229`
[1] "ENSMUSG00000025173"

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
%%R
rxns <- rxn2ensembls.nls %>% names()
print(length(rxns))
print(rxns)

 [325] "R-MMU-383313"  "R-MMU-420123"  "R-MMU-421416"  "R-MMU-422048" 
 [329] "R-MMU-508205"  "R-MMU-508217"  "R-MMU-749448"  "R-MMU-749452" 
 [333] "R-MMU-9023632" "R-MMU-9023633" "R-MMU-163217"  "R-MMU-6799178"
 [337] "R-MMU-6799179" "R-MMU-6799191" "R-MMU-6799196" "R-MMU-6799197"
 [341] "R-MMU-6799202" "R-MMU-6799203" "R-MMU-6800868" "R-MMU-1433273"
 [345] "R-MMU-1433418" "R-MMU-1433428" "R-MMU-1433451" "R-MMU-1433454"
 [349] "R-MMU-1433456" "R-MMU-1433488" "R-MMU-1433506" "R-MMU-1433514"
 [353] "R-MMU-1433542" "R-MMU-1470009" "R-MMU-1470010" "R-MMU-1470012"
 [357] "R-MMU-1472121" "R-MMU-1562641" "R-MMU-202165"  "R-MMU-202168" 
 [361] "R-MMU-202174"  "R-MMU-202214"  "R-MMU-202216"  "R-MMU-202233" 
 [365] "R-MMU-202245"  "R-MMU-202248"  "R-MMU-202291"  "R-MMU-202307" 
 [369] "R-MMU-202344"  "R-MMU-205234"  "R-MMU-205238"  "R-MMU-205244" 
 [373] "R-MMU-205319"  "R-MMU-210289"  "R-MMU-210291"  "R-MMU-2395412"
 [377] "R-MMU-2395436" "R-MMU-2395801" "R-MMU-2396594" "R-MMU-2424480"
 [381]

[1861] "R-MMU-5362459" "R-MMU-5362549" "R-MMU-5362551" "R-MMU-5362553"
[1865] "R-MMU-5632649" "R-MMU-5632652" "R-MMU-5632654" "R-MMU-9759844"
[1869] "R-MMU-9815762" "R-NUL-5362792" "R-MMU-109638"  "R-MMU-112381" 
[1873] "R-MMU-113402"  "R-MMU-113407"  "R-MMU-113409"  "R-MMU-6803523"
[1877] "R-MMU-72095"   "R-MMU-75095"   "R-MMU-77095"   "R-MMU-9012315"
[1881] "R-MMU-9012319" "R-MMU-9838035" "R-MMU-9838289" "R-MMU-8853734"
[1885] "R-MMU-8853737" "R-MMU-8853753" "R-MMU-8853755" "R-MMU-8853762"
[1889] "R-MMU-8853774" "R-MMU-8853792" "R-MMU-8853793" "R-MMU-8853801"
[1893] "R-MMU-8854897" "R-MMU-8854899" "R-MMU-8854905" "R-MMU-8854908"
[1897] "R-MMU-8854981" "R-MMU-8855508" "R-MMU-8855564" "R-MMU-8855617"
[1901] "R-MMU-8855747" "R-MMU-8876190" "R-MMU-8876615" "R-MMU-8877612"
[1905] "R-MMU-556760"  "R-MMU-202241"  "R-MMU-202331"  "R-MMU-202375" 
[1909] "R-MMU-213407"  "R-MMU-2396561" "R-MMU-2396606" "R-MMU-2424481"
[1913] "R-MMU-2424485" "R-MMU-2730833" "R-MMU-2730841" "R-MMU-2730847"
[1917]

[2321] "R-MMU-5696220" "R-MMU-6794347" "R-MMU-6794355" "R-MMU-114256" 
[2325] "R-MMU-6805507" "R-MMU-904830"  "R-MMU-917891"  "R-MMU-390281" 
[2329] "R-MMU-9033235" "R-MMU-9033236" "R-MMU-199895"  "R-MMU-442724" 
[2333] "R-MMU-9029434" "R-MMU-2160851" "R-MMU-964737"  "R-MMU-964825" 
[2337] "R-MMU-964830"  "R-MMU-156723"  "R-MMU-984648"  "R-MMU-71046"  
[2341] "R-MMU-5688786" "R-MMU-5691004" "R-MMU-3221864" "R-MMU-8863858"
[2345] "R-MMU-8863914" "R-MMU-8951595" "R-MMU-901047"  "R-NUL-1236962"
[2349] "R-MMU-350578"  "R-MMU-210426"  "R-MMU-210430"  "R-MMU-372505" 
[2353] "R-MMU-372529"  "R-MMU-374899"  "R-MMU-374922"  "R-MMU-380574" 
[2357] "R-MMU-380869"  "R-MMU-380901"  "R-MMU-380905"  "R-MMU-388824" 
[2361] "R-MMU-3322001" "R-MMU-3322003" "R-MMU-3322005" "R-MMU-3322025"
[2365] "R-MMU-3322041" "R-MMU-3322057" "R-MMU-3781018" "R-MMU-3781023"
[2369] "R-MMU-3781024" "R-MMU-5336443" "R-MMU-5666197" "R-MMU-114264" 
[2373] "R-MMU-114275"  "R-MMU-114284"  "R-MMU-114307"  "R-MMU-139917" 
[2377]

[2781] "R-MMU-5683801" "R-MMU-5684052" "R-MMU-5684071" "R-MMU-5693551"
[2785] "R-MMU-5693566" "R-MMU-69891"   "R-MMU-109857"  "R-MMU-109860" 
[2789] "R-MMU-109863"  "R-MMU-112342"  "R-MMU-5674366" "R-MMU-5674373"
[2793] "R-MMU-5674496" "R-NUL-451681"  "R-MMU-9617595" "R-MMU-8869607"
[2797] "R-MMU-8869627" "R-MMU-1912357" "R-MMU-1980122" "R-MMU-1980125"
[2801] "R-MMU-2066712" "R-MMU-1297338" "R-MMU-5218916" "R-MMU-5687121"
[2805] "R-MMU-9021050" "R-MMU-9021072" "R-MMU-5658231" "R-MMU-5658435"
[2809] "R-MMU-936897"  "R-MMU-9684068" "R-MMU-1237096" "R-MMU-1299507"
[2813] "R-MMU-111925"  "R-MMU-163422"  "R-MMU-177275"  "R-MMU-177284" 
[2817] "R-MMU-265682"  "R-MMU-381644"  "R-MMU-381713"  "R-MMU-432232" 
[2821] "R-MMU-5218854" "R-MMU-5610710" "R-MMU-5610720" "R-MMU-5610749"
[2825] "R-MMU-5624473" "R-MMU-5682101" "R-MMU-5687088" "R-MMU-8951727"
[2829] "R-MMU-9010662" "R-MMU-913996"  "R-MMU-992708"  "R-NUL-5610740"
[2833] "R-MMU-1678708" "R-MMU-2090043" "R-MMU-997237"  "R-MMU-8953917"
[2837]

[3241] "R-MMU-380930"  "R-MMU-380949"  "R-MMU-447074"  "R-MMU-5631941"
[3245] "R-MMU-5665868" "R-MMU-5672008" "R-MMU-5672011" "R-MMU-5673768"
[3249] "R-MMU-5675868" "R-MMU-5679205" "R-MMU-5679239" "R-MMU-6805470"
[3253] "R-MMU-9613545" "R-MMU-936895"  "R-MMU-1237038" "R-MMU-1247665"
[3257] "R-MMU-425482"  "R-MMU-69299"   "R-MMU-74213"   "R-MMU-5691154"
[3261] "R-MMU-199206"  "R-MMU-199219"  "R-MMU-429581"  "R-MMU-373714" 
[3265] "R-MMU-373724"  "R-MMU-373732"  "R-MMU-373734"  "R-MMU-373747" 
[3269] "R-MMU-451377"  "R-MMU-451756"  "R-MMU-451757"  "R-MMU-8981534"
[3273] "R-MMU-6781797" "R-MMU-6781814" "R-MMU-113504"  "R-MMU-174425" 
[3277] "R-MMU-174427"  "R-MMU-174452"  "R-MMU-68913"   "R-MMU-68950"  
[3281] "R-MMU-69053"   "R-MMU-69063"   "R-MMU-69068"   "R-MMU-69074"  
[3285] "R-MMU-69116"   "R-MMU-69127"   "R-MMU-69152"   "R-MMU-9668597"
[3289] "R-MMU-183084"  "R-MMU-183094"  "R-MMU-2197698" "R-MMU-3928643"
[3293] "R-MMU-418850"  "R-MMU-442572"  "R-MMU-448947"  "R-MMU-5218812"
[3297]

...

[6301] "R-NUL-9005752" "R-MMU-204392"  "R-MMU-204465"  "R-MMU-204485" 
[6305] "R-MMU-204600"  "R-MMU-373867"  "R-MMU-373875"  "R-MMU-375133" 
[6309] "R-MMU-375135"  "R-MMU-9645220" "R-MMU-449115"  "R-MMU-8983294"
[6313] "R-MMU-8983300" "R-MMU-8983307" "R-MMU-8983313" "R-MMU-8983335"
[6317] "R-MMU-8983356" "R-MMU-8983384" "R-MMU-9007126" "R-MMU-9007137"
[6321] "R-MMU-9009867" "R-MMU-158399"  "R-MMU-8852266" "R-MMU-8852481"
[6325] "R-MMU-9021306" "R-MMU-9650473" "R-MMU-373074"  "R-MMU-443978" 
[6329] "R-MMU-5578910" "R-MMU-8936214" "R-MMU-5433074" "R-MMU-9753944"
[6333] "R-MMU-975449"  "R-MMU-1471314" "R-MMU-5621348" "R-MMU-9717206"
[6337] "R-MMU-9728723" "R-MMU-9728748" "R-MMU-70454"   "R-MMU-70481"  
[6341] "R-MMU-162830"  "R-NUL-3876071" "R-MMU-5205652" "R-MMU-2161500"
[6345] "R-MMU-549304"  "R-MMU-561072"  "R-MMU-374896"  "R-MMU-374919" 
[6349] "R-MMU-549129"  "R-MMU-549279"  "R-MMU-549322"  "R-MMU-561054" 
[6353] "R-MMU-9794523" "R-MMU-9795207" "R-MMU-8848215" "R-MMU-5669023"
[6357]

[6761] "R-MMU-8855232" "R-MMU-8855259" "R-MMU-193691"  "R-MMU-2730894"
[6765] "R-MMU-448703"  "R-MMU-5660660" "R-MMU-5660666" "R-MMU-8865237"
[6769] "R-MMU-9604480" "R-MMU-1299304" "R-MMU-5250209" "R-MMU-5250210"
[6773] "R-MMU-5250217" "R-MMU-5672027" "R-MMU-3697882" "R-MMU-8940300"
[6777] "R-MMU-1296348" "R-MMU-139905"  "R-NUL-3139045" "R-MMU-1855177"
[6781] "R-MMU-445075"  "R-MMU-1482695" "R-MMU-75886"   "R-MMU-5334050"
[6785] "R-MMU-212007"  "R-MMU-5362525" "R-MMU-2404134" "R-MMU-2404135"
[6789] "R-MMU-2453863" "R-MMU-2453876" "R-MMU-508040"  "R-MMU-2045911"
[6793] "R-MMU-8985822" "R-MMU-8985829" "R-MMU-9010235" "R-MMU-5624130"
[6797] "R-MMU-5624133" "R-MMU-5638004" "R-MMU-5638006" "R-MMU-5638007"
[6801] "R-MMU-5638016" "R-MMU-141186"  "R-MMU-374909"  "R-MMU-379382" 
[6805] "R-MMU-379395"  "R-MMU-9677917" "R-MMU-8948139" "R-MMU-8948143"
[6809] "R-MMU-8948146" "R-MMU-8959571" "R-MMU-8959573" "R-MMU-2162066"
[6813] "R-MMU-74248"   "R-MMU-9754916" "R-MMU-9755078" "R-NUL-3221949"
[6817]

[7221] "R-MMU-186511"  "R-MMU-201595"  "R-MMU-8858428" "R-MMU-448708" 
[7225] "R-MMU-8987012" "R-MMU-5660470" "R-MMU-5660533" "R-MMU-8875902"
[7229] "R-MMU-9675215" "R-MMU-9676254" "R-MMU-9676257" "R-MMU-9676264"
[7233] "R-MMU-9676268" "R-MMU-373738"  "R-MMU-443774"  "R-MMU-6786652"
[7237] "R-MMU-6806831" "R-MMU-5617178" "R-MMU-70606"   "R-MMU-8874979"
[7241] "R-MMU-6783939" "R-MMU-451033"  "R-MMU-70982"   "R-MMU-109903" 
[7245] "R-MMU-373339"  "R-MMU-140664"  "R-MMU-140686"  "R-MMU-9015122"
[7249] "R-MMU-376141"  "R-MMU-428888"  "R-MMU-9706287" "R-MMU-3222237"
[7253] "R-MMU-8959798" "R-MMU-176606"  "R-MMU-9635801" "R-MMU-1861621"
[7257] "R-MMU-199161"  "R-MMU-381026"  "R-MMU-381135"  "R-MMU-381186" 
[7261] "R-MMU-420818"  "R-MMU-389639"  "R-MMU-8939880" "R-MMU-8939892"
[7265] "R-MMU-446204"  "R-MMU-2562594" "R-MMU-194632"  "R-MMU-194689" 
[7269] "R-MMU-71260"   "R-MMU-139970"  "R-MMU-5692315" "R-MMU-3343700"
[7273] "R-MMU-140791"  "R-MMU-140806"  "R-MMU-140870"  "R-MMU-140872" 
[7277]

[7681] "R-MMU-2168881" "R-MMU-2168888" "R-MMU-2203516" "R-MMU-8856189"
[7685] "R-MMU-1250256" "R-MMU-1250264" "R-MMU-1250280" "R-MMU-5676940"
[7689] "R-MMU-1638104" "R-MMU-189439"  "R-MMU-190141"  "R-MMU-6797803"
[7693] "R-MMU-6797810" "R-MMU-203567"  "R-MMU-5690873" "R-MMU-5690911"
[7697] "R-MMU-6797956" "R-MMU-432074"  "R-MMU-507869"  "R-MMU-507871" 
[7701] "R-MMU-445714"  "R-MMU-5696197" "R-MMU-5696491" "R-MMU-6781953"
[7705] "R-MMU-8932276" "R-MMU-446372"  "R-MMU-399936"  "R-MMU-8874204"
[7709] "R-MMU-8874849" "R-MMU-1861788" "R-MMU-6811428" "R-MMU-6811429"
[7713] "R-MMU-9693214" "R-MMU-4085021" "R-MMU-4085028" "R-MMU-9629869"
[7717] "R-MMU-5676637" "R-MMU-163664"  "R-MMU-200423"  "R-MMU-174368" 
[7721] "R-MMU-9644869" "R-MMU-6788798" "R-MMU-9603987" "R-MMU-9845538"
[7725] "R-MMU-444120"  "R-MMU-429567"  "R-MMU-8856525" "R-MMU-8867667"
[7729] "R-MMU-5655483" "R-MMU-5684806" "R-MMU-5684836" "R-MMU-68947"  
[7733] "R-MMU-9734730" "R-MMU-192341"  "R-MMU-193533"  "R-MMU-2066781"
[7737]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [11]:
%%R
rxn_knn_misclass_rate.nls <- list()
rxn_knn_ari.nls <- list()
rxn_knn_ecount.nls <- list()
rxn_pca.nls <- list()
count <- 0

In [12]:
%%R
toi_indices <- seq(1,length(tcdd_dose_detail.vec))
print(toi_indices)

 [1]  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
[26] 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50
[51] 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68


In [13]:
%%R
tcdd_dose_detail_vec_tis_of_interest <-
   tcdd_dose_detail.vec[toi_indices]
print(tcdd_dose_detail_vec_tis_of_interest)

 [1]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
[26]  0  0  0  0  0  0  0  0  0 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
[51] 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30


In [14]:
%%R
vst.count.mtx.tis_of_interest <- vst.count.mtx[, toi_indices]
print(vst.count.mtx.tis_of_interest)

                   SRR7817650 SRR7817612 SRR3593524 SRR7817684 SRR7817697
ENSMUSG00000000001  15.372724  15.489787  14.859455  15.613249  15.278217
ENSMUSG00000000028   7.061426   6.441777   6.706037   6.708989   6.744838
ENSMUSG00000000049  20.478603  20.160850  20.540259  20.467407  20.266653
ENSMUSG00000000058   7.314136   7.794396   7.694075   7.461696   7.609270
ENSMUSG00000000085  10.524518   9.901466  10.184849   9.384417  10.169379
ENSMUSG00000000088  16.429167  16.780332  15.763619  16.599944  16.634191
ENSMUSG00000000120   8.115043   9.351231   8.234509   8.251219   9.077773
ENSMUSG00000000126   6.650515   6.834930   6.445354   6.201722   5.725062
ENSMUSG00000000127   9.230645   8.868388   8.922328   8.467632   8.996591
ENSMUSG00000000142   8.064714   9.417602  12.349300   9.872623   9.416145
ENSMUSG00000000149  16.522736  15.891855  14.998966  16.409305  15.802221
ENSMUSG00000000154  15.777150  15.241950  15.140269  15.594260  15.412980
ENSMUSG00000000168  14.933049  14.1700

ENSMUSG00000001105  12.627223  13.466402  12.685703  13.309612  13.217799
ENSMUSG00000001119   8.726783   8.480903   9.810306   8.716684   8.330397
ENSMUSG00000001123  18.139502  18.459609  18.721873  17.965824  18.339035
ENSMUSG00000001127  15.078423  14.775688  15.047391  14.690127  14.954005
ENSMUSG00000001128  10.502362  12.013853  12.127633   9.654355  11.188177
ENSMUSG00000001131   4.597905   5.410626   5.437735   4.621016   4.599270
ENSMUSG00000001143  10.744173  10.846048  11.798626  11.483234  11.340138
ENSMUSG00000001155  19.223565  18.866063  18.440261  18.798601  18.824139
ENSMUSG00000001158  10.762853   9.981374  11.345098  10.341116  10.402739
ENSMUSG00000001173   9.853309   8.998622   9.952122   9.020527   8.797913
ENSMUSG00000001175  16.960501  17.321548  16.678289  17.384844  17.463738
ENSMUSG00000001211  16.869619  16.952602  17.295571  17.325975  16.978269
ENSMUSG00000001225   4.597905   5.403519   4.598087   4.621016   4.599270
ENSMUSG00000001228   6.650515   6.6775

ENSMUSG00000002249  10.507301  10.484751  10.610871  10.596517  10.920083
ENSMUSG00000002250   9.167863  14.078148  11.511520  11.945153  14.227988
ENSMUSG00000002257   5.437146   5.866745   6.526594   6.027621   5.725062
ENSMUSG00000002279  13.107674  13.149799  13.019631  13.085614  13.360514
ENSMUSG00000002289  15.082668  11.774076  15.124838  14.055484  14.794589
ENSMUSG00000002297   8.617341   7.859083   6.648152   8.065329   7.362610
ENSMUSG00000002307  11.555890  10.737399  11.765101  11.055959  10.645842
ENSMUSG00000002308   8.309094   8.708628   9.073249   8.940010   8.552666
ENSMUSG00000002324   6.341147   7.097146   7.571010   6.606430   7.295125
ENSMUSG00000002325  12.821951  13.252465  13.808760  12.964339  12.932149
ENSMUSG00000002326  11.016040  11.119352  11.429118  10.838455  11.208617
ENSMUSG00000002346  15.439236  15.887100  16.186755  16.225443  15.991950
ENSMUSG00000002365  13.820036  13.249876  13.595554  13.646567  13.489277
ENSMUSG00000002379  13.541798  13.5870

ENSMUSG00000003379   6.555280   6.580237   6.836556   5.813605   6.542770
ENSMUSG00000003402  15.962679  15.775635  16.380129  16.110249  16.170405
ENSMUSG00000003411   5.719533   6.485076   5.919640   5.512192   6.459560
ENSMUSG00000003418   4.597905   5.410626   4.598087   4.621016   4.599270
ENSMUSG00000003421  11.873938  12.238351  11.553688  11.177600  11.630728
ENSMUSG00000003429  13.089987  14.326527  14.207247  14.507928  14.557208
ENSMUSG00000003435  14.399474  15.093854  15.069473  15.238604  15.130348
ENSMUSG00000003436   4.597905   4.589698   4.598087   4.621016   4.599270
ENSMUSG00000003437  12.968238  12.672308  13.204123  12.645000  12.736167
ENSMUSG00000003444   8.026495   7.926248   8.474349   9.144382   8.923615
ENSMUSG00000003452   4.597905   5.681873   5.905417   5.512192   4.599270
ENSMUSG00000003458  13.981447  13.451570  14.148193  13.743260  13.547088
ENSMUSG00000003464  15.725352  15.668952  16.610490  15.577866  15.645380
ENSMUSG00000003476   6.555280   6.3990

ENSMUSG00000004654   4.597905   4.589698   5.384563   4.621016   4.599270
ENSMUSG00000004655  13.047232  14.828214  15.576793  14.477946  13.855859
ENSMUSG00000004665   8.196439   8.867800   8.066974   7.786283   8.858925
ENSMUSG00000004667  12.870285  12.927820  12.076703  12.309850  12.292715
ENSMUSG00000004668   4.597905   5.410626   4.598087   4.621016   4.599270
ENSMUSG00000004677  13.438853  13.225195  12.515482  12.979921  13.082013
ENSMUSG00000004709   5.536362   6.036987   6.080585   6.201722   5.441563
ENSMUSG00000004730   8.707398  11.847106  10.399291   8.286027   9.588373
ENSMUSG00000004768   9.867972   9.678331   9.164549   9.513732   9.242855
ENSMUSG00000004771  14.588570  14.355342  13.915885  14.183474  13.874925
ENSMUSG00000004788  11.282263  12.012922  11.265534  11.545768  11.788652
ENSMUSG00000004789  17.151698  17.168482  16.779965  17.361832  17.184233
ENSMUSG00000004791   5.719533   5.410626   4.598087   4.621016   4.599270
ENSMUSG00000004815  13.479834  13.4395

...

ENSMUSG00000003153   7.249482   7.085878   6.888524
ENSMUSG00000003161  12.300952  11.668735  11.996913
ENSMUSG00000003184  12.804505  13.745893  13.647254
ENSMUSG00000003200  11.083643  12.344931  12.522536
ENSMUSG00000003206   6.818532   7.647298   7.243074
ENSMUSG00000003208   8.184784   8.637915   8.125833
ENSMUSG00000003226  16.733887  15.341830  15.832192
ENSMUSG00000003227   5.459387   5.382711   4.565089
ENSMUSG00000003228   9.921074   9.225193   9.988204
ENSMUSG00000003233  11.918626  12.430601  11.993481
ENSMUSG00000003235  12.685737  13.157249  13.584494
ENSMUSG00000003269  10.282104  10.238052  10.245260
ENSMUSG00000003271   5.317794   5.840469   4.565089
ENSMUSG00000003279   5.117857   5.375788   4.565089
ENSMUSG00000003283   9.419252  10.803974   9.855911
ENSMUSG00000003299  10.631798  12.480886  12.082554
ENSMUSG00000003308  13.684898  14.708348  14.864071
ENSMUSG00000003309   9.917641   9.774252  11.121307
ENSMUSG00000003316  14.416340  14.131237  13.898086
ENSMUSG00000

ENSMUSG00000004885   5.117857   5.992319   4.565089
ENSMUSG00000004892   5.117857   4.581033   4.565089
ENSMUSG00000004895  10.170474  10.712506  10.677516
ENSMUSG00000004902   6.481491   6.360727   6.415641
ENSMUSG00000004929  10.804093  12.035945  12.126455
ENSMUSG00000004933   7.780612   8.531633   8.064712
ENSMUSG00000004934   9.512329  10.959716  11.100219
ENSMUSG00000004936  12.915793  13.168591  13.590932
ENSMUSG00000004937  14.010814  14.729883  14.579730
ENSMUSG00000004939   4.496816   4.581033   4.565089
ENSMUSG00000004947   8.251870   8.655613   8.151220
ENSMUSG00000004948   4.496816   4.581033   5.331615
ENSMUSG00000004951   6.768311   6.691555   6.947727
ENSMUSG00000004952   9.032921  10.903474   9.352505
ENSMUSG00000004961   4.496816   4.581033   4.565089
ENSMUSG00000004980  18.051169  17.408518  17.686180
ENSMUSG00000004988   5.117857   5.516263   5.331615
ENSMUSG00000004996  10.763098  12.598366  12.610137
ENSMUSG00000005034  13.508027  12.929261  13.072043
ENSMUSG00000

ENSMUSG00000006498  14.487252  14.712143  15.070779
ENSMUSG00000006517   7.799366   7.738661   8.319175
ENSMUSG00000006519  12.126069  12.828315  11.590073
ENSMUSG00000006522  16.141980  17.512247  17.157942
ENSMUSG00000006538   6.391576   6.646273   6.326195
ENSMUSG00000006542   6.065080   4.581033   6.377629
ENSMUSG00000006567  11.508905  11.319416  11.706636
ENSMUSG00000006574   4.496816   4.581033   5.331615
ENSMUSG00000006576   7.127935   7.199074   7.153436
ENSMUSG00000006585   8.520125  10.195309   9.707816
ENSMUSG00000006589  13.315146  13.663375  14.120820
ENSMUSG00000006599  13.628753  13.852650  13.699422
ENSMUSG00000006611  13.754875  14.567728  14.472375
ENSMUSG00000006641   9.048029   9.279095   9.161889
ENSMUSG00000006649   5.661947   4.581033   5.760834
ENSMUSG00000006676  15.227858  15.542626  15.428846
ENSMUSG00000006678   8.663313   8.301072   8.646706
ENSMUSG00000006699  16.591426  15.922002  16.162614
ENSMUSG00000006705  10.343667  10.802886  10.346117
ENSMUSG00000

ENSMUSG00000010122   8.591183  10.275539  11.468465
ENSMUSG00000010142   7.472949   7.314987   7.282141
ENSMUSG00000010311   4.496816   4.581033   4.565089
ENSMUSG00000010376  13.466277  14.238763  14.010180
ENSMUSG00000010392  13.300315  13.041884  13.127454
ENSMUSG00000010406  10.964156  12.030482  11.998552
ENSMUSG00000010453  14.749950  14.838046  14.557846
ENSMUSG00000010461   6.463405   6.233575   6.831813
ENSMUSG00000010601   8.238077  11.167662  11.756987
ENSMUSG00000010607   9.747282  10.239721  10.424936
ENSMUSG00000010608  15.682182  14.842064  14.659126
ENSMUSG00000010651  17.738339  18.461711  19.447279
ENSMUSG00000010660   7.062576   6.889690   6.693179
ENSMUSG00000010663  10.152863  11.369160  12.194033
ENSMUSG00000010796   5.117857   4.581033   4.565089
ENSMUSG00000010797   9.148063   9.949829   8.399157
ENSMUSG00000010803   4.496816   4.581033   4.565089
ENSMUSG00000010830   6.607692   7.310782   7.064175
ENSMUSG00000010911   8.701189   8.983981   9.051645
ENSMUSG00000

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
%%R
training_indices <-
   caret::createDataPartition(
      tcdd_dose_detail_vec_tis_of_interest,
      times = 1,
      p = 1.0, # no data will be held out when set to "1.0"
      list = FALSE
   )
print(training_indices)

      Resample1
 [1,]         1
 [2,]         2
 [3,]         3
 [4,]         4
 [5,]         5
 [6,]         6
 [7,]         7
 [8,]         8
 [9,]         9
[10,]        10
[11,]        11
[12,]        12
[13,]        13
[14,]        14
[15,]        15
[16,]        16
[17,]        17
[18,]        18
[19,]        19
[20,]        20
[21,]        21
[22,]        22
[23,]        23
[24,]        24
[25,]        25
[26,]        26
[27,]        27
[28,]        28
[29,]        29
[30,]        30
[31,]        31
[32,]        32
[33,]        33
[34,]        34
[35,]        35
[36,]        36
[37,]        37
[38,]        38
[39,]        39
[40,]        40
[41,]        41
[42,]        42
[43,]        43
[44,]        44
[45,]        45
[46,]        46
[47,]        47
[48,]        48
[49,]        49
[50,]        50
[51,]        51
[52,]        52
[53,]        53
[54,]        54
[55,]        55
[56,]        56
[57,]        57
[58,]        58
[59,]        59
[60,]        60
[61,]        61
[62,]   

In [16]:
%%R
vst.count.mtx.train <-
   vst.count.mtx.tis_of_interest[, training_indices] #9/10ths of data
vst.count.mtx.test  <-
   vst.count.mtx.tis_of_interest[, -training_indices] #1/10th of data

print(vst.count.mtx.train)

                   SRR7817650 SRR7817612 SRR3593524 SRR7817684 SRR7817697
ENSMUSG00000000001  15.372724  15.489787  14.859455  15.613249  15.278217
ENSMUSG00000000028   7.061426   6.441777   6.706037   6.708989   6.744838
ENSMUSG00000000049  20.478603  20.160850  20.540259  20.467407  20.266653
ENSMUSG00000000058   7.314136   7.794396   7.694075   7.461696   7.609270
ENSMUSG00000000085  10.524518   9.901466  10.184849   9.384417  10.169379
ENSMUSG00000000088  16.429167  16.780332  15.763619  16.599944  16.634191
ENSMUSG00000000120   8.115043   9.351231   8.234509   8.251219   9.077773
ENSMUSG00000000126   6.650515   6.834930   6.445354   6.201722   5.725062
ENSMUSG00000000127   9.230645   8.868388   8.922328   8.467632   8.996591
ENSMUSG00000000142   8.064714   9.417602  12.349300   9.872623   9.416145
ENSMUSG00000000149  16.522736  15.891855  14.998966  16.409305  15.802221
ENSMUSG00000000154  15.777150  15.241950  15.140269  15.594260  15.412980
ENSMUSG00000000168  14.933049  14.1700

ENSMUSG00000001105  12.627223  13.466402  12.685703  13.309612  13.217799
ENSMUSG00000001119   8.726783   8.480903   9.810306   8.716684   8.330397
ENSMUSG00000001123  18.139502  18.459609  18.721873  17.965824  18.339035
ENSMUSG00000001127  15.078423  14.775688  15.047391  14.690127  14.954005
ENSMUSG00000001128  10.502362  12.013853  12.127633   9.654355  11.188177
ENSMUSG00000001131   4.597905   5.410626   5.437735   4.621016   4.599270
ENSMUSG00000001143  10.744173  10.846048  11.798626  11.483234  11.340138
ENSMUSG00000001155  19.223565  18.866063  18.440261  18.798601  18.824139
ENSMUSG00000001158  10.762853   9.981374  11.345098  10.341116  10.402739
ENSMUSG00000001173   9.853309   8.998622   9.952122   9.020527   8.797913
ENSMUSG00000001175  16.960501  17.321548  16.678289  17.384844  17.463738
ENSMUSG00000001211  16.869619  16.952602  17.295571  17.325975  16.978269
ENSMUSG00000001225   4.597905   5.403519   4.598087   4.621016   4.599270
ENSMUSG00000001228   6.650515   6.6775

ENSMUSG00000002249  10.507301  10.484751  10.610871  10.596517  10.920083
ENSMUSG00000002250   9.167863  14.078148  11.511520  11.945153  14.227988
ENSMUSG00000002257   5.437146   5.866745   6.526594   6.027621   5.725062
ENSMUSG00000002279  13.107674  13.149799  13.019631  13.085614  13.360514
ENSMUSG00000002289  15.082668  11.774076  15.124838  14.055484  14.794589
ENSMUSG00000002297   8.617341   7.859083   6.648152   8.065329   7.362610
ENSMUSG00000002307  11.555890  10.737399  11.765101  11.055959  10.645842
ENSMUSG00000002308   8.309094   8.708628   9.073249   8.940010   8.552666
ENSMUSG00000002324   6.341147   7.097146   7.571010   6.606430   7.295125
ENSMUSG00000002325  12.821951  13.252465  13.808760  12.964339  12.932149
ENSMUSG00000002326  11.016040  11.119352  11.429118  10.838455  11.208617
ENSMUSG00000002346  15.439236  15.887100  16.186755  16.225443  15.991950
ENSMUSG00000002365  13.820036  13.249876  13.595554  13.646567  13.489277
ENSMUSG00000002379  13.541798  13.5870

ENSMUSG00000003379   6.555280   6.580237   6.836556   5.813605   6.542770
ENSMUSG00000003402  15.962679  15.775635  16.380129  16.110249  16.170405
ENSMUSG00000003411   5.719533   6.485076   5.919640   5.512192   6.459560
ENSMUSG00000003418   4.597905   5.410626   4.598087   4.621016   4.599270
ENSMUSG00000003421  11.873938  12.238351  11.553688  11.177600  11.630728
ENSMUSG00000003429  13.089987  14.326527  14.207247  14.507928  14.557208
ENSMUSG00000003435  14.399474  15.093854  15.069473  15.238604  15.130348
ENSMUSG00000003436   4.597905   4.589698   4.598087   4.621016   4.599270
ENSMUSG00000003437  12.968238  12.672308  13.204123  12.645000  12.736167
ENSMUSG00000003444   8.026495   7.926248   8.474349   9.144382   8.923615
ENSMUSG00000003452   4.597905   5.681873   5.905417   5.512192   4.599270
ENSMUSG00000003458  13.981447  13.451570  14.148193  13.743260  13.547088
ENSMUSG00000003464  15.725352  15.668952  16.610490  15.577866  15.645380
ENSMUSG00000003476   6.555280   6.3990

ENSMUSG00000004654   4.597905   4.589698   5.384563   4.621016   4.599270
ENSMUSG00000004655  13.047232  14.828214  15.576793  14.477946  13.855859
ENSMUSG00000004665   8.196439   8.867800   8.066974   7.786283   8.858925
ENSMUSG00000004667  12.870285  12.927820  12.076703  12.309850  12.292715
ENSMUSG00000004668   4.597905   5.410626   4.598087   4.621016   4.599270
ENSMUSG00000004677  13.438853  13.225195  12.515482  12.979921  13.082013
ENSMUSG00000004709   5.536362   6.036987   6.080585   6.201722   5.441563
ENSMUSG00000004730   8.707398  11.847106  10.399291   8.286027   9.588373
ENSMUSG00000004768   9.867972   9.678331   9.164549   9.513732   9.242855
ENSMUSG00000004771  14.588570  14.355342  13.915885  14.183474  13.874925
ENSMUSG00000004788  11.282263  12.012922  11.265534  11.545768  11.788652
ENSMUSG00000004789  17.151698  17.168482  16.779965  17.361832  17.184233
ENSMUSG00000004791   5.719533   5.410626   4.598087   4.621016   4.599270
ENSMUSG00000004815  13.479834  13.4395

...

ENSMUSG00000017264  12.549831  12.932494  12.809984  11.059426  11.503026
ENSMUSG00000017288  14.119502  13.897642  13.374945  13.088329  13.363996
ENSMUSG00000017291  13.939837  14.078960  13.650502  14.562076  14.924179
ENSMUSG00000017300   5.591079   4.554651   4.571662   4.547679   4.535966
ENSMUSG00000017307  11.169898  11.182138  10.916173  11.068739  10.557543
ENSMUSG00000017309  10.635337  11.104422  10.628795   9.850001  11.417909
ENSMUSG00000017311   4.565996   4.554651   4.571662   6.039869   4.535966
ENSMUSG00000017316   4.565996   4.554651   4.571662   4.547679   4.535966
ENSMUSG00000017344  19.192301  19.572761  19.251303  20.877929  20.553732
ENSMUSG00000017376  10.484569  10.217001  10.296798  10.203502  10.644078
ENSMUSG00000017390   6.946986   7.737125   6.886435   9.427309  10.770444
ENSMUSG00000017404  13.880247  13.646102  14.180563  14.635394  14.143433
ENSMUSG00000017412   6.949240   7.396512   7.061939   7.860903   9.321705
ENSMUSG00000017428  15.591304  15.2814

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



  15.618139  15.494311
ENSMUSG00000000154  13.934857  14.515786  14.769528
ENSMUSG00000000168  13.585652  13.055936  13.821711
ENSMUSG00000000171  16.492746  16.708995  17.153774
ENSMUSG00000000182   6.212373   5.382711   5.911007
ENSMUSG00000000183   5.317794   4.581033   4.565089
ENSMUSG00000000184  11.008854  10.121074  10.388670
ENSMUSG00000000197   5.117857   4.581033   4.565089
ENSMUSG00000000214   4.496816   4.581033   4.565089
ENSMUSG00000000215   4.496816   4.581033   4.565089
ENSMUSG00000000216   4.496816   4.581033   4.565089
ENSMUSG00000000223   5.889004   5.382711   4.565089
ENSMUSG00000000247   7.330699   8.082251   8.170854
ENSMUSG00000000253   8.610211   8.749202   8.510654
ENSMUSG00000000263   4.496816   4.581033   4.565089
ENSMUSG00000000275  14.537253  14.570513  14.414994
ENSMUSG00000000276   9.746453   9.151180   8.626817
ENSMUSG00000000290  12.630813  13.761394  12.020661
ENSMUSG00000000296   8.721125   7.995401   8.996394
ENSMUSG00000000301  14.809371  16.204751 

ENSMUSG00000001751  10.735342  11.422303  11.524617
ENSMUSG00000001755  14.844641  15.390960  16.190163
ENSMUSG00000001761   9.400977  10.206816  10.084953
ENSMUSG00000001767  11.217622  10.534238  11.074522
ENSMUSG00000001768  11.774332  11.457644  11.254762
ENSMUSG00000001773   5.728157   5.651424   4.565089
ENSMUSG00000001786  12.244619  12.868712  13.503866
ENSMUSG00000001794  15.804943  16.550752  16.052002
ENSMUSG00000001804   4.496816   4.581033   4.565089
ENSMUSG00000001827   5.459387   5.382711   5.587453
ENSMUSG00000001833  14.961847  14.013275  14.205318
ENSMUSG00000001847  15.508314  15.452879  15.869115
ENSMUSG00000001855  11.907886  11.847825  11.631908
ENSMUSG00000001865   5.024606   4.581033   4.565089
ENSMUSG00000001870  10.455623   9.972685   9.358001
ENSMUSG00000001891  19.369215  18.374719  19.184464
ENSMUSG00000001901   4.496816   4.581033   4.565089
ENSMUSG00000001918  13.217603  14.280718  13.579828
ENSMUSG00000001924  16.445911  16.487304  16.767859
ENSMUSG00000

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [17]:
%%R
tcdd_dose_detail.vec.train <-
   tcdd_dose_detail_vec_tis_of_interest[training_indices]
tcdd_dose_detail.vec.test <-
   tcdd_dose_detail_vec_tis_of_interest[-training_indices]

# print(gtex_tissue_detail.vec.train)
print(length(unique(tcdd_dose_detail.vec.train)))

In [18]:
%%R
saveRDS(tcdd_dose_detail.vec.train,file=paste(OUT_DIR,"tcdd_dose_detail_vec_train030_time_cross_50.Rds",sep=""))

In [19]:
%%R
cv_fold_indices <- caret::createFolds(tcdd_dose_detail.vec.train,
                                      k = N_FOLDS)
print(cv_fold_indices)

In [20]:
%%R
binary_tcdd_dose_annotations <- unique(tcdd_dose_detail.vec)
print(binary_tcdd_dose_annotations)

In [21]:
%%R
full_rxn_pca_results.nls <- list()
rxn_id_2_result_file_idx.nls <- list()

In [22]:
%%R
n_rxns <- length(rxns)
print(n_rxns)
result_idx <- 1
for(rxn_id_idx in seq(1:n_rxns)){
   rxn_id <- rxns[rxn_id_idx]
   rxn_pca <-
      prcomp(t(vst.count.mtx.train[rxn2ensembls.nls[[rxn_id]], ]), scale. = T)
   full_rxn_pca_results.nls[[rxn_id]] <- rxn_pca
   rxn_id_2_result_file_idx.nls[[rxn_id]] <- result_idx
   rxn_pca.nls[[rxn_id]] <-
      rxn_pca$x[, 1] # 1st principal component coordinate within this reaction-space for each sample
   if(mod(rxn_id_idx,100) == 0){
      print(paste("Processed ",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      flush.console()
   }
   if(mod(rxn_id_idx,1000) == 0){
      print(paste("Storing PCA objects containing reactions ",rxn_id_idx-1000,
                  "-",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      saveRDS(full_rxn_pca_results.nls,
              paste(OUT_DIR, "full_rxn_pca_results_nls",rxn_id_idx-1000,
                    "-",rxn_id_idx,"030_time_cross_50.Rds", sep=""))
      full_rxn_pca_results.nls <- list()
      gc()
      result_idx <- result_idx + 1
   }
}

[1] "Processed 9500 of 9825 reactions (96.7%)..."
[1] "Processed 9600 of 9825 reactions (97.7%)..."
[1] "Processed 9700 of 9825 reactions (98.7%)..."
[1] "Processed 9800 of 9825 reactions (99.8%)..."


In [23]:
%%R
# store remaining PCA objects and removing from RAM
saveRDS(rxn_id_2_result_file_idx.nls,
        paste(OUT_DIR,"rxn_id_2_result_file_idx_nls030_time_cross_50.Rds",sep=""))
saveRDS(full_rxn_pca_results.nls,
        paste(OUT_DIR, "full_rxn_pca_results_nls030_time_cross_50.Rds", sep=""))
rm(full_rxn_pca_results.nls)
gc()

           used  (Mb) gc trigger  (Mb) max used  (Mb)
Ncells  6951248 371.3   13077551 698.5 11518746 615.2
Vcells 15068042 115.0   25842636 197.2 25842633 197.2


In [24]:
%%R
# compare informaction content of below files with pca plots or similar
saveRDS(rxn_pca.nls, paste(OUT_DIR, "rxn_pca_nls030_time_cross_50.Rds", sep = ""))
saveRDS(vst.count.mtx.train,
        paste(OUT_DIR, "vst_count_mtx_train030_time_cross_50.Rds", sep = ""))

In [25]:
%%R
for (cv_fold in names(cv_fold_indices)) {
  cur_cv_fold_indices <- cv_fold_indices[[cv_fold]]
  print(cur_cv_fold_indices)
  print("----------")
}

[1]  6 13 24 41 51 64
[1] "----------"
[1] 18 26 31 32 39 47 57
[1] "----------"
[1]  4  7 28 46 53 59 61
[1] "----------"
[1]  8 19 22 40 50 62
[1] "----------"
[1] 10 21 27 36 38 49 65
[1] "----------"
[1]  2 15 25 35 63 66
[1] "----------"
[1]  3 14 30 34 43 56 68
[1] "----------"
[1]  9 16 17 33 54 55 60
[1] "----------"
[1]  1 20 23 42 45 48 58
[1] "----------"
[1]  5 11 12 29 37 44 52 67
[1] "----------"


In [ ]:
%%R
formatted_annotations <- format(binary_tcdd_dose_annotations, nsmall = 2)

formatted_annotations <- trimws(formatted_annotations)

binary_tcdd_dose_annotations <- formatted_annotations

print(binary_tcdd_dose_annotations)
for (annotation in binary_tcdd_dose_annotations) {
    print(annotation)
}

[1] "0.00"  "30.00"
[1] "0.00"
[1] "30.00"


In [ ]:
%%R
# main loop
for (rxn_id_idx in seq(1:length(rxns))) {
   rxn_id <- rxns[rxn_id_idx]
   ensembl_ids <- rxn2ensembls.nls[[rxn_id]]
   
   mean_misclass_rate <- list()
   sum_ari <- 0

   for (cv_fold in names(cv_fold_indices)) {
      cur_cv_fold_indices <- cv_fold_indices[[cv_fold]]
      
      vst.count.mtx.train.cv_train <-
         vst.count.mtx.train[, -cur_cv_fold_indices] # 4/5ths of training features
      vst.count.mtx.train.cv_test <-
         vst.count.mtx.train[, cur_cv_fold_indices] # 1/5th of training features
      
      tcdd_dose_detail.vec.train.cv_train <-
         tcdd_dose_detail.vec.train[-cur_cv_fold_indices] # 4/5ths of training labels
      tcdd_dose_detail.vec.train.cv_test <-
         tcdd_dose_detail.vec.train[cur_cv_fold_indices] # 1/5th of training labels
      
      binary_tcdd_dose_detail_vec.test.cv_test_list <- list()
      for (tissue_annotation in binary_tcdd_dose_annotations) {
         binary_tcdd_dose_detail_vec.test.cv_test_list[[as.character(tissue_annotation)]] <-
            (as.character(tcdd_dose_detail.vec.train.cv_test) == as.character(tissue_annotation))
      }
      
      cv_train.expr_mat <-
         t(vst.count.mtx.train.cv_train[ensembl_ids, ])
      cv_test.expr_mat <-
         t(vst.count.mtx.train.cv_test[ensembl_ids, ])
      
      rxn_knn_calls <- class::knn(train = cv_train.expr_mat,
                                  test = cv_test.expr_mat,
                                  cl = tcdd_dose_detail.vec.train.cv_train)
      
      # calculate & store adjusted rand index
      cur_ari <- pdfCluster::adj.rand.index(rxn_knn_calls,
                                            tcdd_dose_detail.vec.train.cv_test)
      sum_ari <- cur_ari + sum_ari
      
      # for each tissue, calculate misclassification rate
      for (tissue_annotation in binary_tcdd_dose_annotations) {
         cur_rxn_knn_calls <- (rxn_knn_calls == tissue_annotation)
            
         # calculate misclassification rate (https://stat.ethz.ch/pipermail/r-help/2011-September/288885.html)
         tab <- table(cur_rxn_knn_calls,
                      binary_tcdd_dose_detail_vec.test.cv_test_list[[tissue_annotation]])
        # print(tab)
         cur_misclass_rate <- 1 - sum(diag(tab)) / sum(tab)
        # print(paste("Misclass rate = 1 - ",sum(diag(tab))," / ",sum(tab)," = ", cur_misclass_rate,"...",sep=""))
         sum_misclass_rate <- cur_misclass_rate
         if (!is.null(mean_misclass_rate[[tissue_annotation]])) {
           sum_misclass_rate <- sum_misclass_rate + mean_misclass_rate[[tissue_annotation]]
         }
         mean_misclass_rate[[tissue_annotation]] <- sum_misclass_rate
      }
   }
   for(tissue_annotation in binary_tcdd_dose_annotations){
      mean_misclass_rate[[tissue_annotation]] <- (mean_misclass_rate[[tissue_annotation]] / N_FOLDS)
   }
   mean_ari <- sum_ari / N_FOLDS
   ecount <- length(ensembl_ids)
   
   rxn_knn_misclass_rate.nls[[rxn_id]] <- mean_misclass_rate
   assign("mean_misclass_rate",NULL,envir = .GlobalEnv)
   
   rxn_knn_ari.nls[[rxn_id]] <- mean_ari
   rxn_knn_ecount.nls[[rxn_id]] <- ecount
   
   count <- count + 1
   if (mod(count, 10) == 0) {
      print(
         paste(
            "Last RXN_ID = ",
            rxn_id,
            ": Last ARI = ",
            mean_ari,
            ": Last ECOUNT = ",
            ecount,
            ": Last Lung MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Lung"]],
            ": Last Uterus MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Uterus"]],
            ". Now ",
            round(1.0 - count / length(rxns),3) * 100,
            "% remaining..."
         )
      )
      flush.console()
   }
}

[1] "Last RXN_ID =  R-MMU-749456 : Last ARI =  1 : Last ECOUNT =  178 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-9021601 : Last ARI =  1 : Last ECOUNT =  24 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-3108203 : Last ARI =  0.941666666666667 : Last ECOUNT =  15 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-9838627 : Last ARI =  0.572232283830222 : Last ECOUNT =  4 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-193679 : Last ARI =  0.158838646338646 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-194544 : Last ARI =  -0.056849871282861 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-3247837 : Last

[1] "Last RXN_ID =  R-MMU-450580 : Last ARI =  0.941666666666667 : Last ECOUNT =  12 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-6791222 : Last ARI =  0.825 : Last ECOUNT =  61 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-9696268 : Last ARI =  1 : Last ECOUNT =  41 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-1912394 : Last ARI =  1 : Last ECOUNT =  9 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-8936481 : Last ARI =  1 : Last ECOUNT =  77 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-9677051 : Last ARI =  0.941666666666667 : Last ECOUNT =  10 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-5336403 : Last ARI =  0.883333333333333 

[1] "Last RXN_ID =  R-MMU-983156 : Last ARI =  1 : Last ECOUNT =  242 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-174119 : Last ARI =  1 : Last ECOUNT =  18 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-174224 : Last ARI =  1 : Last ECOUNT =  18 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-3788708 : Last ARI =  0.905405405405405 : Last ECOUNT =  23 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-927830 : Last ARI =  0.943243243243243 : Last ECOUNT =  11 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-448955 : Last ARI =  0.825 : Last ECOUNT =  6 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89 % remaining..."
[1] "Last RXN_ID =  R-MMU-200652 : Last ARI =  -0.00783162226461196 : L

[1] "Last RXN_ID =  R-MMU-420980 : Last ARI =  -0.0447685592015489 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  84.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-9013004 : Last ARI =  1 : Last ECOUNT =  16 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  84.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-9818576 : Last ARI =  0.000356561954500101 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  84 % remaining..."
[1] "Last RXN_ID =  R-MMU-2029273 : Last ARI =  0.883333333333333 : Last ECOUNT =  71 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  83.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-8850356 : Last ARI =  0.874099099099099 : Last ECOUNT =  11 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  83.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-1363303 : Last ARI =  0.874099099099099 : Last ECOUNT =  9 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  83.7 % remaining..."
[1] "Last RXN_ID =  R

[1] "Last RXN_ID =  R-MMU-187506 : Last ARI =  -0.10412012012012 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  79.1 % remaining..."
[1] "Last RXN_ID =  R-NUL-9625422 : Last ARI =  0.381381381381381 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  79 % remaining..."
[1] "Last RXN_ID =  R-MMU-1449633 : Last ARI =  1 : Last ECOUNT =  5 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  78.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-211910 : Last ARI =  1 : Last ECOUNT =  12 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  78.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-3928646 : Last ARI =  0.499624624624625 : Last ECOUNT =  12 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  78.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-8848484 : Last ARI =  0.760585585585586 : Last ECOUNT =  5 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  78.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-9022185 : Last ARI

...

[1] "Last RXN_ID =  R-MMU-73792 : Last ARI =  0.0985093340763444 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  23.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-198275 : Last ARI =  0.650975975975976 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  23 % remaining..."
[1] "Last RXN_ID =  R-MMU-114683 : Last ARI =  0.58506006006006 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  22.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-75090 : Last ARI =  0.00783136983136984 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  22.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-77342 : Last ARI =  0.00193391250092281 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  22.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-449937 : Last ARI =  0.0217428967428967 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  22.5 % remaining..."
[1] "Last RXN_ID

[1] "Last RXN_ID =  R-MMU-139854 : Last ARI =  0.941666666666667 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  18 % remaining..."
[1] "Last RXN_ID =  R-MMU-198983 : Last ARI =  0.346546546546547 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  17.9 % remaining..."
[1] "Last RXN_ID =  R-NUL-2128982 : Last ARI =  -0.087902077335067 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  17.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-9757010 : Last ARI =  0.883333333333333 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  17.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-9653592 : Last ARI =  0.069981981981982 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  17.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-9668464 : Last ARI =  -0.0754396148726046 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  17.5 % remaining..."
[1] "Last R

[1] "Last RXN_ID =  R-MMU-6799545 : Last ARI =  0.0295824670824671 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  12.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-76386 : Last ARI =  1 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  12.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-9012036 : Last ARI =  0.0438546794216897 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  12.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-975633 : Last ARI =  0.427262313860252 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  12.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-9027532 : Last ARI =  0.699099099099099 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  12.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-917888 : Last ARI =  0.404034188034188 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  12.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-68

[1] "Last RXN_ID =  R-MMU-9629360 : Last ARI =  0.326436488034426 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  7.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-6809624 : Last ARI =  0.466187341187341 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  7.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-192312 : Last ARI =  0.759009009009009 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  7.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-8879117 : Last ARI =  0.336232386232386 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  7.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-139909 : Last ARI =  -0.097881287314277 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  7.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-8851979 : Last ARI =  0.503688740286678 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  7.3 % remaining..."
[1] "Last RXN_ID 

[1] "Last RXN_ID =  R-MMU-9622513 : Last ARI =  0.0501621621621622 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  2.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-5661125 : Last ARI =  0.891151202749141 : Last ECOUNT =  14 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  2.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-9695949 : Last ARI =  0.322897897897898 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  2.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-1855172 : Last ARI =  0.38033033033033 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  2.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-6807055 : Last ARI =  0.891151202749141 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  2.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-1462039 : Last ARI =  0.50016021175815 : Last ECOUNT =  9 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  2.2 % remaining..."
[1] "Last RXN_ID

In [28]:
%%R
saveRDS(
   rxn_knn_misclass_rate.nls,
   paste(OUT_DIR, "toi_rxn_knn_misclass_rate_nls030_time_cross_50.Rds", sep = "")
)
saveRDS(rxn_knn_ari.nls,
        paste(OUT_DIR, "toi_rxn_knn_ari_nls030_time_cross_50.Rds", sep = ""))
saveRDS(rxn_knn_ecount.nls,
        paste(OUT_DIR, "toi_rxn_knn_ecount_nls030_time_cross_50.Rds", sep = ""))

In [29]:
%%R
d <- data.frame(
   RXN_ID = names(rxn2ensembls.nls),
   MISCLASS = unlist(rxn_knn_misclass_rate.nls),
   ARI = unlist(rxn_knn_ari.nls),
   ECOUNT = unlist(rxn_knn_ecount.nls)
)
print(d)
saveRDS(d, paste(OUT_DIR, "toi_summary_df030_time_cross_50.Rds", sep = ""))

                           RXN_ID MISCLASS           ARI ECOUNT
R-MMU-170666.0.00    R-MMU-170666        0  0.9324324324     14
R-MMU-170666.30.00   R-MMU-170671        0  0.9324324324     14
R-MMU-170671.0.00    R-MMU-170674        0  0.9324324324     14
R-MMU-170671.30.00   R-MMU-170686        0  0.9324324324     14
R-MMU-170674.0.00    R-MMU-380073        0  1.0000000000    182
R-MMU-170674.30.00   R-MMU-392187        0  1.0000000000     22
R-MMU-170686.0.00    R-MMU-392195        0  1.0000000000     22
R-MMU-170686.30.00   R-MMU-392202        0  1.0000000000     22
R-MMU-380073.0.00    R-MMU-749454        0  1.0000000000    178
R-MMU-380073.30.00   R-MMU-749456        0  1.0000000000    178
R-MMU-392187.0.00    R-MMU-751001        0  1.0000000000     21
R-MMU-392187.30.00  R-MMU-8964229        0  1.0000000000      5
R-MMU-392195.0.00   R-MMU-8964242        0  1.0000000000      5
R-MMU-392195.30.00  R-MMU-8964329        0  1.0000000000      5
R-MMU-392202.0.00   R-MMU-8982012       

R-MMU-205099.30.00  R-MMU-5654224        0  1.0000000000     12
R-MMU-205112.0.00   R-MMU-5654392        0  1.0000000000     20
R-MMU-205112.30.00  R-MMU-5654397        0  0.9416666667     16
R-MMU-205117.0.00   R-MMU-5654399        0  0.9416666667     16
R-MMU-205117.30.00  R-MMU-5654402        0  1.0000000000     20
R-MMU-209532.0.00   R-MMU-5654404        0  0.9416666667     16
R-MMU-209532.30.00  R-MMU-5654406        0  0.9416666667     18
R-MMU-209566.0.00   R-MMU-5654407        0  0.9416666667     16
R-MMU-209566.30.00  R-MMU-5654408        0  0.9416666667     12
R-MMU-9012761.0.00  R-MMU-5654409        0  0.9416666667     12
R-MMU-9012761.30.00 R-MMU-5654413        0  1.0000000000     17
R-MMU-3247837.0.00  R-MMU-5654416        0  0.9416666667     15
R-MMU-3247837.30.00 R-MMU-5654418        0  1.0000000000     14
R-MMU-3247840.0.00  R-MMU-5654422        0  1.0000000000     14
R-MMU-3247840.30.00 R-MMU-5654423        0  1.0000000000     16
R-MMU-3247843.0.00  R-MMU-5654425       

R-MMU-5654224.30.00 R-MMU-9011082        0  0.6680280796      2
R-MMU-5654392.0.00  R-MMU-9011144        0 -0.0304054054      3
R-MMU-5654392.30.00 R-MMU-9011146        0 -0.0304054054      3
R-MMU-5654397.0.00  R-MMU-9753028        0  0.6106119196      5
R-MMU-5654397.30.00 R-MMU-9753126        0  0.6106119196      5
R-MMU-5654399.0.00   R-MMU-514604        0  0.4125465465      2
R-MMU-5654399.30.00  R-MMU-975389        0  0.7406156156      4
R-MMU-5654402.0.00  R-MMU-1169394        0  0.8343944460      5
R-MMU-5654402.30.00 R-MMU-1169398        0  0.5655405405      5
R-MMU-5654404.0.00  R-MMU-1169402        0  0.8740990991      6
R-MMU-5654404.30.00 R-MMU-1169403        0  0.6185435435      4
R-MMU-5654406.0.00  R-MMU-1169405        0  0.7173423423      5
R-MMU-5654406.30.00 R-MMU-1169406        0  1.0000000000     12
R-MMU-5654407.0.00  R-MMU-1678842        0  1.0000000000     13
R-MMU-5654407.30.00 R-MMU-1678843        0  0.6590090090      5
R-MMU-5654408.0.00  R-MMU-5653754       

R-MMU-5654659.30.00 R-MMU-2424487        0  0.9416666667     22
R-MMU-5654662.0.00   R-MMU-388814        0  0.7019620136      6
R-MMU-5654662.30.00  R-MMU-388829        0  1.0000000000      9
R-MMU-5654663.0.00   R-MMU-388831        0  1.0000000000      8
R-MMU-5654663.30.00  R-MMU-388832        0  1.0000000000      9
R-MMU-5654664.0.00   R-MMU-388833        0  1.0000000000     17
R-MMU-5654664.30.00  R-MMU-389158        0  1.0000000000      9
R-MMU-5654667.0.00   R-MMU-389348        0  0.5385135135      8
R-MMU-5654667.30.00  R-MMU-389350        0  0.6331081081      8
R-MMU-5654669.0.00   R-MMU-389352        0  0.5952702703      7
R-MMU-5654669.30.00  R-MMU-389354        0  0.5952702703      7
R-MMU-5654672.0.00   R-MMU-389381        0  0.7035385901      6
R-MMU-5654672.30.00  R-MMU-389758        0  0.8833333333     22
R-MMU-5654673.0.00   R-MMU-389759        0  0.7590090090      7
R-MMU-5654673.30.00  R-MMU-389762        0  0.7006756757      5
R-MMU-5654677.0.00   R-MMU-434836       

R-MMU-9011082.30.00 R-MMU-9692139        0  0.1465090090      2
R-MMU-9011144.0.00  R-MMU-9692145        0  0.8833333333      2
R-MMU-9011144.30.00 R-MMU-9692156        0  0.2161501502      4
R-MMU-9011146.0.00  R-MMU-9692157        0  0.2161501502      4
R-MMU-9011146.30.00 R-MMU-9692268        0  0.1488096344      2
R-MMU-9753028.0.00   R-MMU-450434        0  0.1033153153      1
R-MMU-9753028.30.00  R-MMU-450466        0  0.9432432432     45
R-MMU-9753126.0.00   R-MMU-450551        0  1.0000000000      7
R-MMU-9753126.30.00  R-MMU-450580        0  0.9416666667     12
R-MMU-514604.0.00     R-MMU-72103        0  0.9416666667     43
R-MMU-514604.30.00  R-MMU-9770119        0  0.9416666667     69
R-MMU-975389.0.00   R-MMU-9770141        0  1.0000000000    123
R-MMU-975389.30.00  R-MMU-9770142        0  1.0000000000    132
R-MMU-1169394.0.00  R-MMU-9770145        0  1.0000000000    145
R-MMU-1169394.30.00 R-MMU-9770236        0  1.0000000000    119
R-MMU-1169398.0.00  R-MMU-6798748       

...

R-MMU-9766551.0.00  R-MMU-9731590        0  0.3477578094      1
R-MMU-9766551.30.00 R-MMU-9731613        0  0.3477578094      1
R-MMU-442422.0.00   R-MMU-9731632        0  0.3477578094      1
R-MMU-442422.30.00   R-MMU-426520        0  0.2353704220      1
R-MMU-8874364.0.00  R-MMU-5578712        0  0.6126876877      2
R-MMU-8874364.30.00 R-MMU-5578965        0  0.2594695211      2
R-MMU-8936519.0.00  R-MMU-5578966        0  0.2594695211      2
R-MMU-8936519.30.00 R-MMU-9023912        0  0.2353704220      1
R-MMU-1482745.0.00  R-MMU-9820843        0  0.2353704220      1
R-MMU-1482745.30.00 R-MMU-9820863        0  0.2353704220      1
R-MMU-1482771.0.00  R-MMU-1299318        0  0.1902489027      2
R-MMU-1482771.30.00 R-MMU-6786720        0  0.3458809325      1
R-MMU-1482828.0.00  R-MMU-8851538        0  1.0000000000      1
R-MMU-1482828.30.00 R-MMU-8851550        0  1.0000000000      1
R-MMU-1482847.0.00   R-MMU-352052        0  0.4582683199      1
R-MMU-1482847.30.00 R-MMU-6803789       

R-MMU-5696119.0.00   R-MMU-193821        0  0.6898648649      1
R-MMU-5696119.30.00  R-MMU-193824        0  0.6898648649      1
R-MMU-9673827.0.00  R-NUL-1236960        0  0.6393494009      1
R-MMU-9673827.30.00 R-MMU-8874239        0  0.2213303303      2
R-MMU-2872444.0.00  R-MMU-8874322        0  0.0860450450      1
R-MMU-2872444.30.00 R-MMU-8874406        0  0.0860450450      1
R-MMU-204617.0.00   R-MMU-6814086        0  0.4363767614      3
R-MMU-204617.30.00  R-MMU-6814088        0  0.0643147509      2
R-MMU-204647.0.00   R-MMU-5619426        0  0.0544518150      1
R-MMU-204647.30.00  R-MMU-8848023        0  0.7652503018      2
R-MMU-204662.0.00   R-MMU-8848077        0  0.7652503018      2
R-MMU-204662.30.00  R-MMU-8848110        0  0.4996246246      2
R-MMU-1482932.0.00  R-MMU-8848774        0  0.7590090090      2
R-MMU-1482932.30.00 R-MMU-8848776        0  0.7590090090      2
R-MMU-451360.0.00   R-MMU-8848930        0  0.3838588589      1
R-MMU-451360.30.00  R-MMU-8849055       

R-MMU-1254248.0.00  R-MMU-8953339        0  0.3940040556      1
R-MMU-1254248.30.00 R-MMU-5696839        0 -0.0458946853      1
R-MMU-1254251.0.00  R-MMU-6801456        0 -0.0458946853      1
R-MMU-1254251.30.00 R-MMU-8944219        0  0.4418268784      1
R-MMU-2032770.0.00   R-MMU-879594        0  0.4301902418      1
R-MMU-2032770.30.00  R-MMU-211874        0  0.4675025541      1
R-MMU-2032775.0.00   R-MMU-350604        0  0.8864864865      1
R-MMU-2032775.30.00  R-MMU-373717        0  0.0855213461      2
R-MMU-163320.0.00   R-MMU-1482961        0  0.5598448964      1
R-MMU-163320.30.00  R-MMU-1482962        0  0.8416666667      2
R-MMU-8851988.0.00   R-MMU-264418        0  0.0633287133      1
R-MMU-8851988.30.00  R-MMU-264435        0  0.3499349865      2
R-MMU-6813749.0.00  R-MMU-4167511        0  0.4747097613      2
R-MMU-6813749.30.00 R-MMU-5635102        0  0.0007627628      1
R-MMU-71691.0.00    R-MMU-3209159        0  0.5437037553      1
R-MMU-71691.30.00   R-MMU-1471338       

R-MMU-6798474.0.00  R-MMU-8878914        0  0.0517029123      1
R-MMU-6798474.30.00 R-MMU-8858270        0  0.7161512027      1
R-MMU-6798528.0.00  R-MMU-5605262        0 -0.0184280434      1
R-MMU-6798528.30.00  R-MMU-444620        0  0.4717818334      2
R-MMU-9844860.0.00  R-MMU-9009950        0  0.2400929776      1
R-MMU-9844860.30.00 R-MMU-8937744        0  0.2472597598      1
R-MMU-446185.0.00   R-MMU-8937767        0  0.2472597598      1
R-MMU-446185.30.00   R-MMU-914067        0  0.6587938454      2
R-MMU-9706330.0.00  R-MMU-9604980        0  0.8265765766      2
R-MMU-9706330.30.00 R-MMU-9606789        0  0.8157657658      3
R-MMU-8944218.0.00  R-MMU-8870352        0  0.1434042290      1
R-MMU-8944218.30.00 R-MMU-8870354        0  0.1434042290      1
R-MMU-111881.0.00    R-MMU-419428        0  1.0000000000      4
R-MMU-111881.30.00  R-MMU-3296301        0  0.1734334850      2
R-MMU-111883.0.00   R-MMU-9830451        0  0.5101451967      1
R-MMU-111883.30.00   R-MMU-975829       

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [30]:
%%R
pid <- read.table(paste(OUT_DIR,"tcdd_project_id030_time_cross_50.txt",sep=""),header=FALSE)
sid <- read.table(paste(OUT_DIR,"tcdd_sample_id030_time_cross_50.txt",sep=""), header=FALSE)
print(sid)

           V1
1  SRR7817650
2  SRR7817612
3  SRR3593524
4  SRR7817684
5  SRR7817697
6  SRR3593525
7  SRR7817672
8  SRR7817664
9  SRR4317610
10 SRR7817637
11 SRR7817639
12 SRR7817613
13 SRR1636588
14 SRR7817662
15 SRR1636592
16 SRR4317612
17 SRR7817636
18 SRR7817659
19 SRR7817611
20 SRR7817649
21 SRR1636587
22 SRR7817671
23 SRR7817673
24 SRR7817687
25 SRR7817685
26 SRR7817683
27 SRR7817660
28 SRR1636591
29 SRR7817635
30 SRR1636594
31 SRR1636590
32 SRR7817695
33 SRR7817700
34 SRR3593523
35 SRR3593574
36 SRR7817680
37 SRR7817669
38 SRR7817703
39 SRR7817693
40 SRR7817619
41 SRR4317660
42 SRR7817682
43 SRR7817618
44 SRR7817620
45 SRR7817706
46 SRR7817629
47 SRR3593576
48 SRR4317657
49 SRR7817634
50 SRR1636672
51 SRR7817701
52 SRR4317658
53 SRR7817656
54 SRR7817654
55 SRR3593575
56 SRR7817690
57 SRR3593578
58 SRR3593577
59 SRR1636673
60 SRR7817668
61 SRR7817691
62 SRR7817705
63 SRR7817653
64 SRR3593580
65 SRR1636675
66 SRR4317655
67 SRR7817658
68 SRR7817667


In [ ]:
%%R
library(htmlwidgets)
min_misclass_pca <-
   prcomp(t(vst.count.mtx.train), scale. = T)
pca.d <- data.frame(
   PC1 = min_misclass_pca$x[, 1],
   PC2 = min_misclass_pca$x[, 2],
   PC3 = min_misclass_pca$x[, 3],
   Section = tcdd_dose_detail.vec.train,
   Project_id = pid$V1,
   Sample_id = sid$V1
)

ggplot(pca.d, aes(x = PC1, y = PC2, colour = Section)) +
   geom_point(size = 3) +  
   geom_text(aes(label = paste(Project_id, Sample_id)), vjust = -1, hjust = 1, size = 2) + 
   scale_colour_manual(values = c("0" = "red", "30" = "green")) +  
   theme_bw()

p <- plot_ly(
   data = pca.d,
   x = ~PC1,
   y = ~PC2,
   z = ~PC3,
   type = "scatter3d",
   mode = "markers",
   marker = list(
     color = as.factor(pca.d$Section), 
     colors = c("red", "green"),  
     size = 5
   ),
   text = ~paste("Project ID:", Project_id, "Sample ID:", Sample_id)  
)
saveWidget(p, "plotly_chart030_time_cross_50.html", selfcontained = TRUE)